In [1]:
import sys
sys.path.insert(0, '/home/christo/notebooks/christo/prod/')
sys.path.insert(0, '/home/christo/notebooks/christo/company_maintenance_scripts/')
import re
import pandas as pd
from NedbankSpendPy import  client
import time

In [2]:
from nedbank_companies_addition_lib import maxindexfn, mongo_regex_searchstring
from nedbank_companies_addition_lib import nedbankcompany_class_check, classlookupfn, input_dict_fn, mongo_regex_searchstring_row
from nedbank_companies_addition_lib import  nedbank_companies_db, regex_searchstring

In [ ]:
def client_agg_fn(collection):
    return pd.DataFrame(
    list(collection.aggregate([{'$match':
                                                        {
                                                            'companyname':
                                                            {
                                                                '$exists':True
                                                            }
                                                        }
                                                       },
                                                       {'$project':{'companyname':1,
                                                                    'Dedupegroup':1,
                                                                    'companyindex':1,
                                                                   'TransactionAmount':1}},
                                                       {
                                                           '$group':
                                                           {
                                                               '_id':
                                                               {
                                                                   'companyname':'$companyname',
                                                                   'companyindex':'$companyindex'
                                                               },
                                                               "uniqueclientlist":{"$addToSet":'$Dedupegroup'},
                                                               "transactioncount":
                                                               {
                                                                   '$sum': 1
                                                               },
                                                               "transactionamount":
                                                               {
                                                                   '$sum':'$TransactionAmount'
                                                               }
                                                           }
                                                       },
                                                       {'$project':{'_id':0,'companyname':'$_id.companyname',
                                                                    'companyindex':'$_id.companyindex',
                                                                    'n_uniqueclients':{'$size': "$uniqueclientlist"},
                                                                   'transactioncount':1,
                                                                   'transactionamount':1}}
                                                      ], allowDiskUse=True)))



In [3]:
def company_agg_fn(collection):
    return pd.DataFrame(
    list(collection.aggregate([{'$match':
                                                        {
                                                            'companyname':
                                                            {
                                                                '$exists':True
                                                            }
                                                        }
                                                       },
                                                       {'$project':{
                                                           'companyname':1,
#                                                            'franchisename':1,
                                                           'Dedupegroup':1,
#                                                            'companyindex':1,
                                                           'TransactionAmount':1}},
                                                       {
                                                           '$group':
                                                           {
                                                               '_id':
                                                               {
                                                                   'companyname':'$companyname',
#                                                                    'franchisename':'$franchisename'
                                                               },
                                                               "uniqueclientlist":{"$addToSet":'$Dedupegroup'},
                                                               "transactioncount":
                                                               {
                                                                   '$sum': 1
                                                               },
                                                               "transactionamount":
                                                               {
                                                                   '$sum':'$TransactionAmount'
                                                               }
                                                           }
                                                       },
                                                       {'$project':{'_id':0,'companyname':'$_id.companyname',
                                                                    'franchisename':'$_id.franchisename',
#                                                                     'companyindex':'$_id.companyindex',
                                                                    'n_uniqueclients':{'$size': "$uniqueclientlist"},
                                                                   'transactioncount':1,
                                                                   'transactionamount':1}}
                                                      ], allowDiskUse=True)))



In [4]:
collectionlist=['clientswipes_201701','clientswipes_201702','clientswipes_201703',
                'clientswipes_201704','clientswipes_201705','clientswipes_201706',
                'clientswipes_201707','clientswipes_201708','clientswipes_201709',
                'clientswipes_201710','clientswipes_201711','clientswipes_201712',
                'clientswipes_201801','clientswipes_201802','clientswipes_201803',
                'clientswipes_201804','clientswipes_201805','clientswipes_201806',
                'clientswipes_201807','clientswipes_201808','clientswipes_201809',
                'clientswipes_201810','clientswipes_201811','clientswipes_201812',                
                'clientswipes_201901','clientswipes_201902','clientswipes_201903',
                'clientswipes_201904','clientswipes_201905','clientswipes_201906',
                'clientswipes_201907','clientswipes_201908','clientswipes_201909',
                'clientswipes_201910','clientswipes_201911','clientswipes_201912']

In [5]:
from datetime import datetime 

In [6]:
collectionname=collectionlist[0]
collection=client.behaviourdb[collectionname]
df=franchise_agg_fn(collection)

NameError: name 'franchise_agg_fn' is not defined

In [ ]:
dn=datetime.now()
m=dn.month
d=dn.day
y=dn.year
savestr= f'{y:0}'+f'{m:02}'+f'{d:02}'
df=pd.DataFrame()
for collectionname in collectionlist:
#     print('-----------------------------------------------------------------------------')
    print('===== now doing collection: {collectionname} ====================================='.format(collectionname=collectionname))
    period=collectionname[-6:]
    print('Doing period: {period}'.format(period=period))
    collection=client.behaviourdb[collectionname]
    print('Collect data:')    
    agg_df=client_agg_fn(collection)
    print('Set index:')    
    agg_df.set_index(['companyindex','companyname'],inplace=True)
    print('Rename columns:')        
    agg_df.rename(columns={'transactionamount':'amount_{period}'.format(period=period),
                      'transactioncount':'count_{period}'.format(period=period),
                          'n_uniqueclients':'n_uniqueclients_{period}'.format(period=period)}, inplace=True)
#     agg_df['period']=period
    if df.empty:
        print('The history is still empty!')
        df=agg_df.copy()
    else:
        print('Merge new data with (non-empty) history:')        
        df=df.merge(agg_df, right_index=True,left_index=True, how='outer')
    print('Save (new) history:')
#     df.to_csv(df.to_csv('/home/christo/data/companydb_franchiselevel_aggregated_pivot_{savestr}.csv'.format(savestr=savestr)))
    

In [ ]:
df.to_csv('/home/christo/data/companydb_franchiselevel_aggregated_pivot_{savestr}.csv'.format(savestr=savestr))

In [ ]:
collection = client.lookups['aggregated_company_spend']
df.reset_index(inplace=True)
df['extracted_date']=savestr
collection.insert_many(df.to_dict('records'))

In [ ]:
list(df)

#### Extract distinct franchisenames:

#### - BOLT: